# Sequências Musicais com Python

Nesta aula vamos utilizar o pacote [MIDIUtil](https://pypi.org/project/MIDIUtil/) para produzir arquivos [Midi](https://en.wikipedia.org/wiki/General_MIDI) (Musical Instrument Digital Interface). Utilizaremos algumas das funções do pacote itertools para produzir improvisações musicais simples. 

Na primeira parte explicaremos rapidamente os conceitos de Teoria da Música necessários para entender os exercícios. Algumas referências interessantes para aprender um pouco de música programando:
- [Haskell School of Music](http://euterpea.com/haskell-school-of-music/)
- [Music for Geeks and Nerds](https://pedrokroger.net/mfgan/)

Utilizei [midi2abc](https://manned.org/midi2abc/0b4697e7) e o pacote [abc](https://ctan.org/pkg/abc) para gerar as partituras dos exercícios. 


## Notas
Existem 7 [notas musicais](https://pt.wikipedia.org/wiki/Nota_musical): Do, Re, Mi, Fá, Sol, La, Si

Aqui vamos utilizar o sistema (padrão em ocidente) de cifras que utiliza letras para representar as notas musicais: C D E F G A B

Existem duas alterações ou [acidentes cromáticos](https://pt.wikipedia.org/wiki/Acidente) dessas notas: # (sustenido) e b (bemol). Por exemplo C# denota Dó sustenido e Bb represente Si bemol. Note que C# e Db correspondem à mesma nota (igualmente D# e Eb, etc). 

<img src="img/notas-piano.png">

## Oitavas
A sequência `C D E F G A B` se repete e cada repetição é conhecida como uma oitava. Por exemplo, C0 corresponde ao primeiro Dó no teclado, C1 ao segundo, C2 ao terceiro, etc. C4 normalmente é conhecido como Dó central. 


## Pentagrama
O pentagrama ou [pauta](https://pt.wikipedia.org/wiki/Pauta_(m%C3%BAsica)) possui 5 linhas e 4 espaços. Segue um exemplo simples com a notação utilizada: 
<img src="img/pentagrama.png">


## Duração 
A unidade de medida da música é o tempo. Cada tempo corresponde a uma pulsação. Segue a representação normalmente utilizada: 

- semibreve (a unidade)
- mínima  (1/2 da unidade)
- semínima  (1/4 da unidade)
- colcheia (1/8 da unidade)
- semicolcheia  (1/16 da unidade)
- fusa (1/32 da unidade)
- semifusa (1/64 da unidade)

A fórmula do compasso define a quantidade de notas e o tipo de notas que cada compasso pode ter. Por exemplo, 4/4 representa que cada cada compasso deve ter quatro semínimas e 3/4 define compassos com 3 semínimas. 

<img src="img/44.png">

<img src="img/24.png">



## Tons e semitons 

No formato [Midi](https://pt.wikipedia.org/wiki/MIDI) cada uma das notas do teclado está representada por números entre 0 e 127. Por exemplo, o Dó central (C4) corresponde ao número 60. 

Assim, podemos facilmente transformar notas em números 0-127 utilizando a fórmula 12*oitava + altura (note que cada oitava possui 12 semitons) :

In [2]:
# As notas com bemol
NOTES_FLAT = ['C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B']
# As notas com sustenido
NOTES_SHARP = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

def NoteToMidi(KeyOctave):
    # KeyOctave is formatted like 'C#3'
    # Fonte: https://stackoverflow.com/questions/13926280/musical-note-string-c-4-f-3-etc-to-midi-note-value-in-python
    key = KeyOctave[:-1]  # eg C, Db
    octave = KeyOctave[-1]   # eg 3, 4
    answer = -1

    try:
        if 'b' in key:
            pos = NOTES_FLAT.index(key)
        else:
            pos = NOTES_SHARP.index(key)
    except:
        print('The key is not valid', key)
        return answer

    answer += pos + 12 * (int(octave) + 1) + 1
    return answer

def MidiToNote(valor):
    '''Transformar um valor Midi a Nota'''
    return NOTES_SHARP[valor % 12] + str(valor//12 - 1)

def adicionarOitava(nota, oitava):
    '''Adiciona a oitava: ("Gb",3) --> Gb3 '''
    return nota + str(oitava)

print(f'C4 corresponde a : {NoteToMidi("C4")}')
print(f'62 corresponde a : {MidiToNote(62)}')

C4 corresponde a : 60
62 corresponde a : D4


Note, que aumentar 1 significa subir um semitom:
 - De C a D temos um tom (2 semitons). Igualmente entre D-E, G-A e A-B.
 - Entre E-F e B-C temos 1 semitom. 
 - De  C e C# temos um semitom (similarmente para as outras notas sustenidas)

## Midi
 O formato Midi é uma serie de mensagens que permitem tocar instrumentos musicais (eletrônicos).  As mensagens normalmente indicam uma ação (tocar uma nota) e o tempo no qual essa ação deve ser executada. 
 
 O exemplo a seguir produz um arquivo Midi com a escala de Dó maior

In [3]:
from midiutil import MIDIFile

# --------------------------
track    = 0  #faixa
channel  = 0  # canal
time     = 0  # Beat inicial
tempo    = 60  # BPM (beats/pulsações por minuto)
volume   = 100  # 0-127, (padrão MIDI)
arquivo  = "1.mid"
# --------------------------

# Criar um objeto tipo Midi
MyMIDI = MIDIFile(1)  # Só um track
# No track 0 (o único neste caso), a partir do beat 0 (o primeiro) 
# o tempo será 60 (60 beats por segundo). 
MyMIDI.addTempo(track, time, tempo)

# Escala de Dó maior (4ta oitava)
escalaC = ["C4", "D4", "E4", "F4", "G4", "A4", "B4", "C5"]
# Converter a valores midi
escala = map(NoteToMidi, escalaC)
for i, n in enumerate(escala):
    # Parâmetros: faixa, canal, nota, tempo, duração, volume
    MyMIDI.addNote(track, channel, n, i, 1, volume)
    # Note que i toma os valores 0,1,2,3... Isso significa que, 
    # em cada pulsação teremos uma nota. Como a duração é 1 e o tempo é 60 (beats por minuto), 
    # cada nota terá uma duração de um segundo. 

# Escrever a um arquivo
with open(arquivo, "wb") as output_file:
    MyMIDI.writeFile(output_file)
    
    

Escute o resultado no arquivo <a href="./1.mid">1.mid</a> : Cada segundo (tempo = 60) escutamos uma das notas da escala maior. 

<img src="img/1mid.png">

## Exercício (Escala Maior)
Toda [escala maior](https://pt.wikipedia.org/wiki/Escala_maior) obedece à seguinte ordem:

Tom - Tom - Semitom - Tom - Tom - Tom - Semitom

Note que na escala de C maior, os semitons correspondem a E-F e B-C. 

Um intervalo é a diferença (em número de semitons) entre dois notas. Por exemplo, temos um intervalo de 2 entre C e D. 

Implemente uma função que receba como parâmetro uma nota ("E4", "G#4", etc), uma sequência de intervalos e gere a sequência de sons resultantes. Por exemplo, se passarmos como parâmetro "C4" e a sequência [2,2,1,2,2,2,1] devemos obter a sequência de valores Midi [60, 62, 64, 65, 67, 69, 71, 72] que corresponde exatamente à escala C maior. 


In [ ]:
# Exercício Escala Maior


## Exercício (Chain)
A função [chain(it1,it2,...)](https://docs.python.org/3/library/itertools.html#itertools.chain) do pacote `itertools` constrói  um iterador que primeiro retorna os elementos de it1, depois os elementos de it2, etc. 

A [escala menor](https://pt.wikipedia.org/wiki/Escala_menor) obedece à ordem:
tom, semitom, tom, tom, semitom, tom, tom

Escreva uma função que, dada uma nota N ("E4", "G#4", etc) gere a escala N Maior, depois N menor, depois N menor em ordem reversa e finalmente N Maior em ordem reversa. 

Por exemplo, se N="E4", o resultado deve ser <a href="2.mid">2.mid</a>. Nesse arquivo, utilizei tempo=120 (2 notas por segundo).

<img src="img/2mid.png">

Na escala de Mi maior, Fá e Sol, Dó e Ré devem ser sustenidos para respeitar os intervalos: E F# (1 tom), F# G# (1 tom), G# A (1 semitom), etc. Já na escala menor: E F# (1 tom), F# G (1 semitom), etc. Note o [Bequadro](https://pt.wikipedia.org/wiki/Acidente) que cancela os sustenidos definidos na armadura (em particular, Sol e Dó deixam de ser sustenidos para respeitar os intervalos da escala menor). 


In [ ]:
# Exercício 2


## Adicionando ritmo
Vamos utilizar uma namedtuple para representar as notas e as suas durações:

In [ ]:
from collections import namedtuple
# Por exemplo NStr("C#4", 2)
NStr = namedtuple("NStr", "nota dur") 
# Por exemplo NNum(60, 2)
NNum = namedtuple("NNum", "nota dur") 

def toNum(seq):
    '''Transforma uma sequência de NStr para NNum '''
    return map(lambda x: NNum(NoteToMidi(x.nota), x.dur), seq) 

Para facilitar as coisas, vamos criar uma função que recebe uma sequência de NNum e gera um arquivo Midi

In [ ]:
def tocar(seq, arquivo, tempo=120, volume = 100, track=0, channel=0):
    '''Cria um arquivo midi com os valores da sequência (de objetos tipo NNum)'''
    MyMIDI = MIDIFile(1)  # Só um track
    MyMIDI.addTempo(0, 0, tempo)
    t = 0
    for n,d in seq:
        MyMIDI.addNote(track, channel, n, t, d, volume)
        t+=d

    with open(arquivo, "wb") as output_file:
        MyMIDI.writeFile(output_file)
    
# Um exemplo simples
l = [NStr("C4",2),
     NStr("D4",1),NStr("D4",1),
     NStr("E4",1/4),NStr("E4",1/4),NStr("E4",1/4),NStr("E4",1/4)]

tocar(toNum(l),"3.mid")

Neste caso, com tempo=120, escutamos em <a href="3.mid">3.mid</a> Dó por 1 segundo, 2 vezes Ré (cada uma de meio segundo) e 4 vezes Mi (cada uma de 1/4 de segundo). 

## Exercício (Repetições na escala)
Generalize o último exemplo para que, dada uma nota n e uma duração d, produza a escala maior de n onde a primeira nota tem uma duração d (e é tocada 1 vez), a segunda com duração d/2 (tocada 2 vezes), a terceira d/4, etc. Por exemplo, com tempo=60, d=4 e n="C4" o resultado seria <a href="4.mid">4.mid</a>

In [ ]:
# Exercício Repetições


## Exercício (Soft Kitty)

Vamos produzir uma versão alternativa de [Soft Kitty](https://www.youtube.com/watch?v=N-qra604RbU).  Para isso, primeiro criamos uma função que gere bools aleatórios. 


In [ ]:
import random
def randomseq():
    '''Gera uma sequência aleatória de bools'''
    while True:
        yield random.choice([True, False])
     
# as Notas de Soft Kitty como tuplas NStr
notas = [NStr(nota='G4', dur=2), NStr(nota='E4', dur=1), NStr(nota='E4', dur=1), 
         NStr(nota='F4', dur=2), NStr(nota='D4', dur=1), NStr(nota='D4', dur=1), 
         NStr(nota='C4', dur=1), NStr(nota='D4', dur=1), NStr(nota='E4', dur=1), NStr(nota='F4', dur=1), 
         NStr(nota='G4', dur=4), 
         NStr(nota='G4', dur=1), NStr(nota='G4', dur=1), NStr(nota='E4', dur=1), NStr(nota='E4', dur=1), 
         NStr(nota='F4', dur=1), NStr(nota='F4', dur=1), NStr(nota='D4', dur=1), NStr(nota='D4', dur=1), 
         NStr(nota='C4', dur=2), NStr(nota='D4', dur=2), 
         NStr(nota='C4', dur=4)]

A ideia é que, por cada valor true dessa sequência aleatória, dividimos a nota correspondente em 2 (de duração d/2) como no exercício anterior. Segue o meu resultado com tempo=120 concatenando a versão original da música e depois tocando a versão modificada. Como estamos utilizando números aleatórios, provavelmente a sua versão não corresponde exatamente com a minha (<a href="5.mid">5.mid</a>). 
<img src="img/5mid.png">

In [ ]:
# Exercício Soft Kitty

## Exercício (Transposição)
A transposição é o processo de modificar a altura de uma nota por um intervalo constante. Por exemplo, a transposição de C em 3 semitons seria D# (que é a mesma coisa que Eb). Escreva uma função que, dada uma sequência de notas (NNum) e um número N de semitons, gere a sequência com essa transposição. A sua função deve trabalhar com sequências de objetos tipo NStr e NNum. Aproveite essa função para escutar primeiro o Soft Kitty original, depois com uma transposição de 8 semitons e depois com uma transposição de -4 semitons (<a href="6.mid">6.mid</a>)

In [ ]:
# Exercício Transposição

## Exercício (Rotação)
Uma rotação consiste em deslocar os elementos de uma lista à direita. Por exemplo, se rotamos 2 vezes a lista [a,b,c,d], obtemos [c,d,a,b]. 


Dado um número N de beats/pulsações, gere uma sequência de listas, cada uma contendo grupos de duração N beats. Por exemplo:
```
l = [NStr(nota='G4', dur=2), NStr(nota='E4', dur=1), NStr(nota='E4', dur=1)]
N = 2
res = [
        [ NStr(nota='G4', dur=2)], #duração = 2
        [NStr(nota='E4', dur=1), NStr(nota='E4', dur=1)] # duração = 1 + 1 = 2
       ]
```

Depois, implemente uma função que rote M elementos os grupos ímpares da sequência resultante da transformação acima. Por exemplo, se utilizarmos esse transformação com a escala do Dó maior, agrupando de 2 em 2 notas, o resultado seria <a href="7.mid">7.mid</a> 

<img src="img/7mid.png">



In [ ]:
# Exercício Rotação

## Exercício (Permutações)
Uma forma simples de improvisação é tocar a mesma sequência de notas permutando seus elementos. Escreva uma função que, dada uma sequência de notas,  gere todas as possíveis permutações dela. Por exemplo, para a sequência `["C4","E4","G4"]` o resultado é <a href="8.mid">8.mid</a>. 

<img src="img/8mid.png">


In [ ]:
# Exercício Permutação

# Harmonia 

Nesta seção vamos combinar sons simultâneos. Para isso, vamos produzir notas em canais diferentes. Segue um exemplo simples que executa G Maior na oitava 3 e C Maior na oitava 4. (Resultado em <a href="9.mid">9.mid</a>).

<img src="img/9mid.png">

In [ ]:
def tocarSimultaneo(seq1, seq2, arquivo, tempo=120, volume = 100, track=0):
    '''Seq1 vai ser tocada no canal 0 e seq 2 no canal 2'''
    MyMIDI = MIDIFile(1)  # Só um track
    MyMIDI.addTempo(0, 0, tempo) 
    canal1 = 0
    canal2 = 1
    t=0
    for n,d in seq1:
        MyMIDI.addNote(track, canal1, n, t, d, volume)
        t+= d
    t=0
    for n,d in seq2:
        MyMIDI.addNote(track, canal2, n, t, d, volume)
        t+= d
        
    with open(arquivo, "wb") as output_file:
        MyMIDI.writeFile(output_file)


CMaior =  ( NNum(x,1) for x in gerarEscalaMaior("C",4) )
GMaior =  ( NNum(x,1) for x in gerarEscalaMaior("G",3) )
tocarSimultaneo(CMaior,GMaior,"9.mid")

## Exercício (Instrumentos)

Para trocar o instrumento utilizado, MIDIFile oferece o método
```
addProgramChange(track, channel, time, program)
```
que modifica, no tempo `time`, o "programa" (instrumento) a ser utilizado. Esse programa é um número de 1 a 128. Veja a lista de instrumentos [aqui](https://en.wikipedia.org/wiki/General_MIDI). 

Note que a função tocarSimultaneo inicializa os dois canais no mesmo tempo (t=0), e utiliza o mesmo volume e instrumento para executar as duas sequências. 

Defina uma named tuple para configurar a execução de cada sequência (ou seja, tempo, volume, instrumento, etc). Também, generalize a função anterior para que receba como parâmetro um número não determinado de sequências (e as suas configurações) e gere o arquivo Midi. Lembre que isso pode ser feito em Python com o operador de desempacotamento:

```
def f(*params):
  '''Recebe 0 ou mais parâmetros'''
  for i in params:
    print(i)

f() # Sem parâmetros
f(1) # com um parâmetro
f(1,2) # com dois parâmetros
``` 
Finalmente, teste a sua solução para fazer um "Canon", isto é, tocar soft kitty transposta  e iniciando em instantes diferentes. Por exemplo, em <a href="10.mid">10.mid</a>, Soft Kitty começa com o piano (volume 100) e, depois de 8 pulsações, começa a mesma música transposta -5 semitons tocada na flauta (74).

In [ ]:
# Exercício Instrumentos


## Exercício (Percussão)

Adicionei uma "batida" (ritmo de fundo) em Soft Kitty. Para isso, você pode utilizar os instrumentos de percussão (113-120). A minha versão: <a href="11.mid">11.mid</a>.


In [ ]:
# Exercício Percuss˜ō


## Exercício (Acompanhamento)
Vamos implementar um acompanhamento simples para Soft Kitty. Um [triade](https://pt.wikipedia.org/wiki/Acorde) é um acorde que resulta da sobreposição de uma nota, sua terça (4 semitons acima) e quinta (7 semitons acima). Por exemplo, o acorde de C resulta em CEG. Vamos criar uma segunda voz que vai executar o acorde principal da primeira nota de cada compasso. Esse acorde deve ser gerado uma oitava abaixo (12 semitons) dessa nota. Meu resultado: <a href="12.mid">12.mid</a>. 


In [ ]:
# Exercício Acompanhamento